In [1]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from charts import stations_map
from settings import *

In [4]:
data_path = DATA_PATH+'Oceanic_Domain/3.1OceanSurfaceSubsurfaceTemperature/Figure3.1/'
xls = pd.ExcelFile(
    data_path+'MeanAnnualSeaSurfaceTemperature_Anomalies_MalinHead.xlsx')
df = pd.read_excel(xls, 'MALIN_Timeseries')
movingAverageTotals = df.temperature.rolling(window=5, center=True).mean() 
df["5 Year Moving Average - Mean"]=movingAverageTotals
df

,year,temperature,Calculated Anomalies,5 year moving average,satellite,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,5 Year Moving Average - Mean
0,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1961,10.73,0.181429,NaN,NaN,10.548571,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1962,10.25,-0.298571,-0.206071,NaN,10.548571,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1963,9.74,-0.808571,-0.236571,NaN,10.548571,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.312
4,1964,10.65,0.101429,-0.316571,NaN,10.548571,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2016,11.27,0.721429,0.583429,NaN,10.548571,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.132
57,2017,11.44,0.891429,NaN,NaN,10.548571,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,2018,10.74,0.191429,NaN,NaN,10.548571,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:

annualTrace = go.Bar(x=df["year"],
                     y=df["Calculated Anomalies"],
                     text=df["temperature"],
                     name='Annual',
                     marker=dict(
                            # color="#214a7b", color used in report
                            color=TIMESERIES_COLOR_SECONDARY,
                            opacity=0.5
                            ),
                     hovertemplate='%{x}<br>' +
                            '<b>Annual</b><br>' +
                            'Total: %{text:.2f}\u00b0C<br>' +
                            'Anomaly: %{y:.2f}\u00b0C<extra></extra>'
                            )
movingAverage = go.Scatter(x=df["year"],
                     y=df["5 year moving average"],
                     text=df["5 Year Moving Average - Mean"],
                     name='5yr Moving Average',
                     mode='lines',  # 'line' is default
                     line_shape='spline',
                     line=dict(
                            # color="#fc0d1b", color used in report
                            color=TIMESERIES_COLOR_PRIMARY,
                            width=2),
                     hovertemplate='%{x}<br>' +
                            '<b>5yr Moving Average</b><br>' +
                            'Total: %{text:.2f}\u00b0C<br>' +
                            'Anomaly: %{y:.2f}\u00b0C<extra></extra>'
                            )
normal = go.Scatter(x=df["year"],
                     y=df["Unnamed: 5"],
                     name='1981-2010 Normal',
                     mode='lines',  # 'line' is default
                     line_shape='spline',
                     line=dict(color="#fdbf2d", #color used in report
                               width=1),
                     hoverinfo='skip',
                            )



In [17]:
figure_3_1 = make_subplots(specs=[[{'secondary_y': True}]])
figure_3_1.add_trace(annualTrace,
            secondary_y=False,)
figure_3_1.add_trace(movingAverage,
            secondary_y=False,)
figure_3_1.add_trace(normal,
            secondary_y=True,)

figure_3_1.update_layout(TIMESERIES_LAYOUT)
figure_3_1.update_yaxes(title_text='Difference (\u00b0C) from 1981-2010 Normal',
                        secondary_y=False,
                        range=[-1, 1],
                        showgrid=False,
                        dtick=0.25,  # dtick sets the distance between ticks
                        tick0=0,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        showspikes=True,
                        # zeroline=True,  # add a zero line
                        # zerolinecolor=TIMESERIES_COLOR_SECONDARY
                        )
figure_3_1.update_yaxes(title_text='Annual SST (\u00b0C)',
                        secondary_y=True,
                        range=[9.5, 11.6],
                        showgrid=False,
                        dtick=0.5,  # dtick sets the distance between ticks
                        tick0=10.6,  # tick0 sets a point to map the other ticks
                        fixedrange=True,
                        )

figure_3_1.update_xaxes(
    title='Year',
    fixedrange=True,
    tickformat='000',  
    showspikes=True,  
    spikethickness=2, 
) 
